In [35]:
import warnings

warnings.filterwarnings(
    "ignore",
)
import os

os.environ["NIXTLA_ID_AS_COL"] = "true"
import numpy as np

np.set_printoptions(suppress=True)
np.random.seed(1)
import random

random.seed(1)
import pandas as pd

pd.set_option("max_colwidth", 100)
pd.set_option("display.precision", 3)
import seaborn as sns
from utilsforecast.plotting import plot_series as plot_series_utils

sns.set_style("whitegrid")
import matplotlib.pyplot as plt

plt.style.use("ggplot")
plt.rcParams.update(
    {
        "figure.figsize": (8, 5),
        "figure.dpi": 100,
        "savefig.dpi": 300,
        "figure.constrained_layout.use": True,
        "axes.titlesize": 12,
        "axes.labelsize": 10,
        "xtick.labelsize": 9,
        "ytick.labelsize": 9,
        "legend.fontsize": 9,
        "legend.title_fontsize": 10,
    }
)

# Define a harmonious palette (8 colors)
ggplot_palette = [
    "#E24A33",  # orange-red
    "#348ABD",  # blue
    "#988ED5",  # purple
    "#777777",  # gray
    "#FBC15E",  # yellow
    "#8EBA42",  # green
    "#FFB5B8",  # pink
    "#17BECF",  # cyan
]

# Apply the palette globally
sns.set_palette(ggplot_palette)

from functools import partial

import matplotlib as mpl
from fpppy.utils import plot_series
from IPython.display import Image, Markdown, display
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from statsforecast import StatsForecast
from statsforecast.models import AutoETS
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import STL, seasonal_decompose
from utilsforecast.evaluation import evaluate
from utilsforecast.feature_engineering import fourier, pipeline, trend
from utilsforecast.losses import mae
from utilsforecast.losses import mape
from utilsforecast.losses import mape as _mape
from utilsforecast.losses import mase, mqloss, quantile_loss, rmse


def mape(df, models, id_col="unique_id", target_col="y"):
    df_mape = _mape(df, models, id_col=id_col, target_col=target_col)
    df_mape.loc[:, df_mape.select_dtypes(include="number").columns] *= 100
    return df_mape


from fpppy.models import LinearRegression
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.preprocessing import SplineTransformer
from statsforecast import StatsForecast
from statsforecast.models import (
    HistoricAverage,
    Naive,
    RandomWalkWithDrift,
    SeasonalNaive,
    SklearnModel,
    WindowAverage,
)
from statsforecast.utils import ConformalIntervals

In [7]:
def recursive_ES(y, t, alpha, l0):
    if t == -1:
        return l0
    else:
        return alpha * y[t] + (1 - alpha) * recursive_ES(y, t - 1, alpha, l0)


def Forecast_ES(y, alpha, l0):
    level = recursive_ES(y, len(y) - 1, alpha, l0)
    return level

In [29]:
def SSE_recursive_ES(y, t, alpha, l0):
    if t == -1:
        return (0, l0)
    else:
        sse, lt_1 = SSE_recursive_ES(y, t - 1, alpha, l0)
        sse += np.square(y[t] - lt_1)
        lt = alpha * y[t] + (1 - alpha) * lt_1
        return (sse, lt)


def SSE_Forecast_ES(y, alpha, l0):
    sse, level = SSE_recursive_ES(y, len(y) - 1, alpha, l0)
    return sse

In [43]:
def objective(params, y):
    alpha, l0 = params
    # Constrain alpha between 0 and 1 by penalizing invalid values
    if not (0 <= alpha <= 1):
        return np.inf
    return SSE_Forecast_ES(y, alpha, l0)


def auto_Forecast_ES(y):
    x0 = [0.5, y[0]]
    to_minimize = partial(objective, y=df.y.values)
    alpha, l0 = minimize(to_minimize, x0)["x"]
    return Forecast_ES(y, alpha, l0)

In [30]:
df = pd.read_csv("../DATA/aus_livestock.csv")
df.ds = pd.to_datetime(df.ds)
df = df[df.unique_id == "Victoria_Pigs"]

In [31]:
Forecast_ES(df.y.values, 0.32176486, 100558.59945071)

95187.0733576038

In [32]:
model = AutoETS(season_length=12, model="ANN")
model.fit(y=df.y.values)
print(model.model_["par"])
print(model.predict(h=1)["mean"])

[     0.32176486             nan             nan             nan
 100558.59945071]
[95187.07336295]


In [34]:
SSE_Forecast_ES(df.y.values, 0.8, 100558.59945071)

59581791626.73232

In [42]:
to_minimize = partial(objective, y=df.y.values)
x0 = [0.5, 90558]
minimize(to_minimize, x0)["x"]

array([     0.32199435, 100540.82011812])

In [46]:
auto_Forecast_ES(df.y.values)

95186.7281539134